# 9. Callbacks
callbacks : 학습 도중에 어떤 이벤트를 일으키는 것
지금까지 실습한 부분도 모두 필요하므로 앞에 지금까지 한 내용을 적어주고, 
callbacks 실습을 시작한다. 

##이번 시간에는 callbacks 중에 tensorboard를 여는 방법을 알아보도록 한다. 

### 1. 모듈 불러오기
tensorflow 2.0에서 jupyter에서 바로 tensorboard를 실행 할 수 있게 되었다. 
> using   %load_ext tensorboard

In [2]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

### 2. Hyperparameter 설정

In [3]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

### 3. model 생성

In [9]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs =inputs, outputs = net, name = 'Basic_CNN')

### 4. model fit

In [11]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate), #Optimization
             loss = 'sparse_categorical_crossentropy', #Loss Function
             metrics = ['accuracy'])#Metrics / Accuracy

### 5. Data Preprocess

In [14]:
train_paths = glob('/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/train/*.png')
test_paths = glob('/Users/jeonghyeonjeong/for github/인공지능_딥러닝_dataset/cifar/test/*.png')

len(train_paths), len(test_paths)

(50000, 10000)

In [15]:
#label의 class 보기
#get_class_name : 파일 명으로 부터 label추출하는 함수
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [16]:
#위의 함수를 사용해서 모든 파일의 label 추출 한 후 unique한 것만 뽑아서 총 class가 뭔지, 몇 개인지 보기
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [44]:
#파일마다 label을 추출하고, => 그 label을 one hot encoding까지 마무리하기
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) #그렇게 onehot해줘서 1이 된 곳의 번호를 반환

In [45]:
#image와 label을 함께 나타내기
def load_image_label(path):
    
    #해당 파일의 이미지 불러고오기
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    #normailzation
    image = tf.cast(image, tf.float32)/255.
    
    #해당 이미지의 label불러오기
    label = get_label(path)
    
    return image, label

In [46]:
#이미지 변형 해주기

def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image, label

In [47]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [48]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [49]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

### 6. Callbacks

In [50]:
starttime = datetime.now()

In [51]:
starttime

datetime.datetime(2020, 2, 23, 14, 14, 51, 50280)

In [52]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
logdir
        #앞에서 from datetime import datetime : 현재시간 사용할 때 사용

'logs/20200223-141452'

In [53]:
#callbacks 불러오기
#: tf.keras.callbacks. 하고 tap 누르면 다양한 게 있는데 여기서는 TensorBoard를 써보도록 한다. 
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir = logdir, 
    write_graph=True,
    write_images=True,
    histogram_freq=1
 )

In [54]:
 %tensorboard --logdir logs --port 8008

#이렇게 하면 여기서 tensorboard 실행 할 수 있고, 여기서 바로 볼 수 있다.
#일단 나는 ERROR: Failed to launch TensorBoard (exited with 1) 에러가떠서 안되는데 일단 밑에서 학습 먼저 진행하고, 이를 나중에 해결 해 보도록한다\\

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/Users/jeonghyeonjeong/anaconda3/bin/tensorboard", line 10, in <module>
    sys.exit(run_main())
  File "/Users/jeonghyeonjeong/anaconda3/lib/python3.6/site-packages/tensorboard/main.py", line 58, in run_main
    default.get_plugins() + default.get_dynamic_plugins(),
  File "/Users/jeonghyeonjeong/anaconda3/lib/python3.6/site-packages/tensorboard/default.py", line 110, in get_dynamic_plugins
    for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins')
  File "/Users/jeonghyeonjeong/anaconda3/lib/python3.6/site-packages/tensorboard/default.py", line 110, in <listcomp>
    for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins')
  File "/Users/jeonghyeonjeong/anaconda3/lib/python3.6/site-packages/pkg_resources/__init__.py", line 2442, in load
    self.require(*args, **kwargs)
  File "/Users/jeonghyeonjeong/anaconda3/lib/python3.6/site-packa

### 7. Training

In [ ]:
steps_per_epoch = len(train_paths)//batch_size
validation_steps = len(test_paths)//batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks = [tensorboard])

Epoch 1/10
 823/1562 [==============>...............] - ETA: 7:12 - loss: 2.0496 - accuracy: 0.2192